In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import zipfile
from tqdm import tqdm

zf = zipfile.ZipFile('drive/MyDrive/DL_dataset/cats_dogs.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

100%|██████████| 25004/25004 [00:28<00:00, 863.91it/s] 


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import numpy as np

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from pytorchtools_st import EarlyStopping

In [8]:
cats_dogs = ImageFolder('cats_dogs/train', transform = transforms.Compose([transforms.Resize([400, 400]), 
                                                                                 transforms.ToTensor()]))

In [13]:
train_size = int(np.floor(len(cats_dogs)*0.8))
test_size = len(cats_dogs) - train_size 

dataset_train, test_val = random_split(cats_dogs, [train_size, test_size]) # Разделение на тренировчоную и вылидационные выборки

train_loader = DataLoader(dataset_train, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_val, batch_size = len(test_val), shuffle = True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = nn.Sequential(nn.Conv2d(3, 20, kernel_size=3, padding=1),
                      nn.BatchNorm2d(20), 
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2),
                      nn.Conv2d(20, 20, kernel_size=3, padding=1),
                      nn.BatchNorm2d(20), 
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2),
                      nn.Flatten(),
                      nn.Linear(20*100*100, 64),
                      nn.ReLU(),
                      nn.Dropout(),
                      nn.Linear(64, 32),
                      nn.ReLU(),
                      nn.Dropout(),
                      nn.Linear(32, 10)
                      ).to(device='cpu')

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

early_stop = EarlyStopping(delta = 0.5, patience = 3, verbose = False) # Ранняя остановка

In [14]:
n_epochs = 500

train_history_loss = list()
# val_history_loss = list()
epochs_history = list()

for epoch in range(1, n_epochs + 1):
  epoch_loss = 0
  # val_epoch_loss = 0

  model.train() # явно указываем обучение
  for X_batch, y_batch in train_loader:
    X_batch, y_batch = X_batch.to(device='cpu'), y_batch.to(device='cpu')
    # forward
    out = model(X_batch)
    # loss
    batch_loss = loss(out, y_batch)
    # backward
    batch_loss.backward()
    # optimization
    optimizer.step()
    optimizer.zero_grad()
    epoch_loss += batch_loss.item()

  train_history_loss.append(epoch_loss/len(train_loader))
  epochs_history.append(epoch)

  if epoch % 2 == 0:
    print(f'Epoch: [{epoch}/{n_epochs}], Train_loss: {train_history_loss[-1]}')

  early_stop(train_history_loss[-1], model)
  
  if early_stop.early_stop:
      print(f"Early stopping on epoch: [{epoch}/{n_epochs}]")
      break

torch.save(model, '/content/drive/MyDrive/DL_dataset/model_cats_dogs.pth')

KeyboardInterrupt: ignored